# Create DA Lookup Table

This notebook generates a lookup table between Census Dissemination Area and municipality.

In [1]:
!pip install geopandas

In [2]:
import pandas as pd
from google.cloud import bigquery
from urllib.request import urlretrieve
import geopandas as gpd

Download and unzip the "relationship file" that describes the connection between ID numbers at various levels of geography, along with the shapefiles for DA, Census Division, and Census Subdivision. The shapefiles are primarily used to identify the city / region.

In [3]:
urlretrieve(url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/dguid-idugd/files-fichiers/2021_98260004.zip", filename = "relationship_file.zip")
urlretrieve(url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lda_000b21a_e.zip", filename = "lda_000b21a_e.zip")
urlretrieve(url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcd_000b21a_e.zip", filename = "lcd_000b21a_e.zip")
urlretrieve(url = "https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/files-fichiers/lcsd000b21a_e.zip", filename = "lcsd000b21a_e.zip")

('lcsd000b21a_e.zip', <http.client.HTTPMessage at 0x7f04c7fd73d0>)

In [4]:
!unzip -o relationship_file.zip
!unzip -d lda_000b21a_e -o lda_000b21a_e.zip
!unzip -d lcd_000b21a_e -o lcd_000b21a_e.zip
!unzip -d lcsd000b21a_e -o lcsd000b21a_e.zip

Archive:  relationship_file.zip
  inflating: 2021_98260004.csv       
Archive:  lda_000b21a_e.zip
  inflating: lda_000b21a_e/lda_000b21a_e.dbf  
  inflating: lda_000b21a_e/lda_000b21a_e.prj  
  inflating: lda_000b21a_e/lda_000b21a_e.shp  
  inflating: lda_000b21a_e/lda_000b21a_e.shx  
  inflating: lda_000b21a_e/lda_000b21a_e.xml  
Archive:  lcd_000b21a_e.zip
  inflating: lcd_000b21a_e/lcd_000b21a_e.dbf  
  inflating: lcd_000b21a_e/lcd_000b21a_e.prj  
  inflating: lcd_000b21a_e/lcd_000b21a_e.shp  
  inflating: lcd_000b21a_e/lcd_000b21a_e.shx  
  inflating: lcd_000b21a_e/lcd_000b21a_e.xml  
Archive:  lcsd000b21a_e.zip
  inflating: lcsd000b21a_e/lcsd000b21a_e.dbf  
  inflating: lcsd000b21a_e/lcsd000b21a_e.prj  
  inflating: lcsd000b21a_e/lcsd000b21a_e.shp  
  inflating: lcsd000b21a_e/lcsd000b21a_e.shx  
  inflating: lcsd000b21a_e/lcsd000b21a_e.xml  


In [5]:
relationship_data = pd.read_csv("./2021_98260004.csv", dtype = str)
relationship_data.head()

,PRDGUID_PRIDUGD,CDDGUID_DRIDUGD,FEDDGUID_CEFIDUGD,CSDDGUID_SDRIDUGD,ERDGUID_REIDUGD,CARDGUID_RARIDUGD,CCSDGUID_SRUIDUGD,DADGUID_ADIDUGD,DBDGUID_IDIDUGD,ADADGUID_ADAIDUGD,DPLDGUID_LDIDUGD,CMAPDGUID_RMRPIDUGD,CMADGUID_RMRIDUGD,CTDGUID_SRIDUGD,POPCTRPDGUID_CTRPOPPIDUGD,POPCTRDGUID_CTRPOPIDUGD
0,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165001,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
1,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165002,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
2,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165006,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
3,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010165,2021S051310010165007,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792
4,2021A000210,2021A00031001,2013A000410007,2021A00051001519,2021S05001010,2021S05011001,2021S05021001519,2021S051210010166,2021S051310010166001,2021S051610010007,NaN,NaN,2021S0503001,2021S05070010170.02,NaN,2021S05100792


In [6]:
relationship_data.describe()

,PRDGUID_PRIDUGD,CDDGUID_DRIDUGD,FEDDGUID_CEFIDUGD,CSDDGUID_SDRIDUGD,ERDGUID_REIDUGD,CARDGUID_RARIDUGD,CCSDGUID_SRUIDUGD,DADGUID_ADIDUGD,DBDGUID_IDIDUGD,ADADGUID_ADAIDUGD,DPLDGUID_LDIDUGD,CMAPDGUID_RMRPIDUGD,CMADGUID_RMRIDUGD,CTDGUID_SRIDUGD,POPCTRPDGUID_CTRPOPPIDUGD,POPCTRDGUID_CTRPOPIDUGD
count,498786,498786,498786,498786,498786,498786,498786,498786,498786,498786,20526,14653,285225,233616,9111,265939
unique,13,293,338,5161,76,72,1757,57936,498786,5433,1685,8,152,6247,8,1026
top,2021A000235,2021A00035915,2013A000447002,2021A00053520005,2021S05003530,2021S05013503,2021S05022466023,2021S051247080104,2021S051310010165001,2021S051647080001,2021A0006590384,2021S050535505,2021S0503462,2021S05078050200.00,2021S0511350616,2021S05100944
freq,137867,15406,9178,13323,36986,38011,13844,289,1,2697,224,10118,36332,624,6211,29311


Save the unprocessed dataset.

In [7]:
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    relationship_data, 
    'row-community-database.unprocessed.census_relationship', 
    job_config=job_config
)  
job.result()

LoadJob<project=row-community-database, location=northamerica-northeast2, id=22a3f02d-67eb-4307-bcfe-14bedaba3ab6>

In [8]:
da_shapefile = gpd.read_file("./lda_000b21a_e/lda_000b21a_e.shp")
cd_shapefile = gpd.read_file("./lcd_000b21a_e/lcd_000b21a_e.shp")
csd_shapefile = gpd.read_file("./lcsd000b21a_e/lcsd000b21a_e.shp")

In [9]:
da_shapefile.head()

,DAUID,DGUID,LANDAREA,PRUID,geometry
0,10010165,2021S051210010165,0.3817,10,"POLYGON ((8978199.943 2146681.889, 8978191.283..."
1,10010166,2021S051210010166,0.0954,10,"POLYGON ((8978655.237 2146599.654, 8978545.646..."
2,10010167,2021S051210010167,0.1265,10,"POLYGON ((8978582.477 2146647.180, 8978545.486..."
3,10010168,2021S051210010168,0.1911,10,"POLYGON ((8978846.749 2147522.749, 8978855.369..."
4,10010169,2021S051210010169,0.0662,10,"POLYGON ((8978347.594 2146972.786, 8978272.194..."


In [10]:
cd_shapefile.head()

,CDUID,DGUID,CDNAME,CDTYPE,LANDAREA,PRUID,geometry
0,1001,2021A00031001,Division No. 1,CDR,9104.5799,10,"MULTIPOLYGON (((8991051.954 2038839.069, 89910..."
1,1002,2021A00031002,Division No. 2,CDR,5915.5695,10,"MULTIPOLYGON (((8887835.569 2084264.706, 88878..."
2,1003,2021A00031003,Division No. 3,CDR,19272.1069,10,"MULTIPOLYGON (((8666126.603 1955816.189, 86661..."
3,1004,2021A00031004,Division No. 4,CDR,7019.9723,10,"MULTIPOLYGON (((8536466.374 1926979.874, 85364..."
4,1005,2021A00031005,Division No. 5,CDR,10293.7618,10,"MULTIPOLYGON (((8528149.243 2085441.331, 85281..."


In [11]:
csd_shapefile.head()

,CSDUID,DGUID,CSDNAME,CSDTYPE,LANDAREA,PRUID,geometry
0,1001101,2021A00051001101,"Division No. 1, Subd. V",SNO,870.8928,10,"MULTIPOLYGON (((8991051.954 2038839.069, 89910..."
1,1001105,2021A00051001105,Portugal Cove South,T,1.0770,10,"POLYGON ((9001504.369 2049856.597, 9001184.977..."
2,1001113,2021A00051001113,Trepassey,T,54.2130,10,"POLYGON ((8992311.551 2054468.074, 8994001.626..."
3,1001120,2021A00051001120,St. Shott's,T,1.0729,10,"POLYGON ((8985240.566 2028560.540, 8984967.974..."
4,1001124,2021A00051001124,"Division No. 1, Subd. U",SNO,742.3781,10,"MULTIPOLYGON (((8995265.971 2098624.091, 89952..."


Identify the relevant information in each dataset. The challenge is that "DGUID" has a different meaning in each dataset, but the "relationship_data" explains how DGUID in one data set relates to DGUID in another.
* The DA List is our definitive list of DAs
* The CSD List gives municipality names
* The CD List gives region names

In [12]:
da_list = da_shapefile[['DAUID', 'DGUID']]
csd_list = csd_shapefile[['DGUID', 'CSDNAME']]
cd_list = cd_shapefile[['DGUID', 'CDNAME']]
relationship_data_unique = relationship_data[['DADGUID_ADIDUGD', 'CSDDGUID_SDRIDUGD', 'CDDGUID_DRIDUGD']].drop_duplicates()

In [13]:
da_list

,DAUID,DGUID
0,10010165,2021S051210010165
1,10010166,2021S051210010166
2,10010167,2021S051210010167
3,10010168,2021S051210010168
4,10010169,2021S051210010169
...,...,...
57927,62080024,2021S051262080024
57928,62080025,2021S051262080025
57929,62080026,2021S051262080026
57930,62080027,2021S051262080027


The steps now are:
* Begin with the DA list
* Use the relationship data to identify the corresponding ID numbers in the CSD and CD datasets
* Use the CSD and CD data to identify the city and region names

In [14]:
da_to_relationship = pd.merge(left = da_list,
                              right = relationship_data_unique,
                              how = "left",
                              left_on = "DGUID",
                              right_on = "DADGUID_ADIDUGD")
da_to_relationship

,DAUID,DGUID,DADGUID_ADIDUGD,CSDDGUID_SDRIDUGD,CDDGUID_DRIDUGD
0,10010165,2021S051210010165,2021S051210010165,2021A00051001519,2021A00031001
1,10010166,2021S051210010166,2021S051210010166,2021A00051001519,2021A00031001
2,10010167,2021S051210010167,2021S051210010167,2021A00051001519,2021A00031001
3,10010168,2021S051210010168,2021S051210010168,2021A00051001519,2021A00031001
4,10010169,2021S051210010169,2021S051210010169,2021A00051001519,2021A00031001
...,...,...,...,...,...
57927,62080024,2021S051262080024,2021S051262080024,2021A00056208073,2021A00036208
57928,62080025,2021S051262080025,2021S051262080025,2021A00056208059,2021A00036208
57929,62080026,2021S051262080026,2021S051262080026,2021A00056208059,2021A00036208
57930,62080027,2021S051262080027,2021S051262080027,2021A00056208081,2021A00036208


In [15]:
da_to_csd = pd.merge(left = da_to_relationship,
                     right = csd_list,
                     how = "left",
                     left_on = "CSDDGUID_SDRIDUGD",
                     right_on = "DGUID").drop(['DGUID_x', 'DGUID_y'], axis = 'columns')
da_to_csd

,DAUID,DADGUID_ADIDUGD,CSDDGUID_SDRIDUGD,CDDGUID_DRIDUGD,CSDNAME
0,10010165,2021S051210010165,2021A00051001519,2021A00031001,St. John's
1,10010166,2021S051210010166,2021A00051001519,2021A00031001,St. John's
2,10010167,2021S051210010167,2021A00051001519,2021A00031001,St. John's
3,10010168,2021S051210010168,2021A00051001519,2021A00031001,St. John's
4,10010169,2021S051210010169,2021A00051001519,2021A00031001,St. John's
...,...,...,...,...,...
57927,62080024,2021S051262080024,2021A00056208073,2021A00036208,Cambridge Bay
57928,62080025,2021S051262080025,2021A00056208059,2021A00036208,Kugluktuk
57929,62080026,2021S051262080026,2021A00056208059,2021A00036208,Kugluktuk
57930,62080027,2021S051262080027,2021A00056208081,2021A00036208,Gjoa Haven


In [16]:
da_to_csd_and_cd = pd.merge(left = da_to_csd,
                            right = cd_list,
                            how = "left",
                            left_on = "CDDGUID_DRIDUGD",
                            right_on = "DGUID").drop(['DGUID'], axis = 'columns')
da_to_csd_and_cd

,DAUID,DADGUID_ADIDUGD,CSDDGUID_SDRIDUGD,CDDGUID_DRIDUGD,CSDNAME,CDNAME
0,10010165,2021S051210010165,2021A00051001519,2021A00031001,St. John's,Division No. 1
1,10010166,2021S051210010166,2021A00051001519,2021A00031001,St. John's,Division No. 1
2,10010167,2021S051210010167,2021A00051001519,2021A00031001,St. John's,Division No. 1
3,10010168,2021S051210010168,2021A00051001519,2021A00031001,St. John's,Division No. 1
4,10010169,2021S051210010169,2021A00051001519,2021A00031001,St. John's,Division No. 1
...,...,...,...,...,...,...
57927,62080024,2021S051262080024,2021A00056208073,2021A00036208,Cambridge Bay,Kitikmeot
57928,62080025,2021S051262080025,2021A00056208059,2021A00036208,Kugluktuk,Kitikmeot
57929,62080026,2021S051262080026,2021A00056208059,2021A00036208,Kugluktuk,Kitikmeot
57930,62080027,2021S051262080027,2021A00056208081,2021A00036208,Gjoa Haven,Kitikmeot


In [17]:
da_to_csd_and_cd.describe()

,DAUID,DADGUID_ADIDUGD,CSDDGUID_SDRIDUGD,CDDGUID_DRIDUGD,CSDNAME,CDNAME
count,57932,57932,57932,57932,57932,57932
unique,57932,57932,5161,293,4937,238
top,10010165,2021S051210010165,2021A00053520005,2021A00033520,Toronto,Toronto
freq,1,1,3743,3743,3743,3743


In [18]:
client = bigquery.Client()
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    da_to_csd_and_cd, 
    'row-community-database.census.DA_to_city_lookup', 
    job_config=job_config
)  
job.result()

LoadJob<project=row-community-database, location=northamerica-northeast2, id=5a950ff2-6691-4a58-87a7-f51f26ea155f>

In [19]:
da_shapefile.head()

,DAUID,DGUID,LANDAREA,PRUID,geometry
0,10010165,2021S051210010165,0.3817,10,"POLYGON ((8978199.943 2146681.889, 8978191.283..."
1,10010166,2021S051210010166,0.0954,10,"POLYGON ((8978655.237 2146599.654, 8978545.646..."
2,10010167,2021S051210010167,0.1265,10,"POLYGON ((8978582.477 2146647.180, 8978545.486..."
3,10010168,2021S051210010168,0.1911,10,"POLYGON ((8978846.749 2147522.749, 8978855.369..."
4,10010169,2021S051210010169,0.0662,10,"POLYGON ((8978347.594 2146972.786, 8978272.194..."


In [20]:
!rm relationship_file.zip
!rm 2021_98260004.csv
!rm -R lcd_000b21a_e
!rm -R lcsd000b21a_e
!rm -R lda_000b21a_e
!rm lcd_000b21a_e.zip
!rm lcsd000b21a_e.zip
!rm lda_000b21a_e.zip